# Metadata

**L1 Taxonomy** - Problem Solving & Algorithms

**L2 Taxonomy** - Recursive Algorithms

**Subtopic** - Finding non-repeating chains with digit factorials

**Use Case** - The number $145$ is well known for the property that the sum of the factorial of its digits is equal to $145$:
$$1! + 4! + 5! = 1 + 24 + 120 = 145.$$
Perhaps less well known is $169$, in that it produces the longest chain of numbers that link back to $169$; it turns out that there are only three such loops that exist:
\begin{align}
&169 \to 363601 \to 1454 \to 169\\
&871 \to 45361 \to 871\\
&872 \to 45362 \to 872
\end{align}
It is not difficult to prove that EVERY starting number will eventually get stuck in a loop. For example,
\begin{align}
&69 \to 363600 \to 1454 \to 169 \to 363601 (\to 1454)\\
&78 \to 45360 \to 871 \to 45361 (\to 871)\\
&540 \to 145 (\to 145)
\end{align}
Starting with $69$ produces a chain of five non-repeating terms, but the longest non-repeating chain with a starting number below one million is sixty terms.
How many chains, with a starting number below one million, contain exactly sixty non-repeating terms?


**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

A positive integer can generate a chain by repeatedly replacing the number with the sum of the factorials of its digits. For example, starting with 87, the next term in the chain is the sum of the factorials of 8 and 7, which is 40320 plus 5040 equals 45360. This process continues until a number repeats, at which point a loop is formed and the chain is considered to have ended.

Every starting number eventually falls into a loop, and the sequence of terms before reaching the loop is called the non-repeating part of the chain. This problem asks for the total count of such starting numbers below one million that produce exactly sixty non-repeating terms before repeating.

You are required to compute how many starting numbers below one million produce a chain of exactly sixty non-repeating terms, where each term is formed by replacing a number with the sum of the factorials of its digits. Once a value repeats, the chain is terminated. Only the non-repeating portion of the chain is counted.

## Input Format

* A single integer `limit` representing the exclusive upper bound on the starting numbers. All starting numbers from 1 up to but not including this limit are considered.
* A single integer `required_length` representing the exact length of the non-repeating chain to be counted.

Both inputs are integers and are guaranteed to be positive.

## Output Format

* Return a single integer representing the number of starting numbers below `limit` that generate a chain of exactly `required_length` non-repeating terms.

## Examples

Example 1:

Input:

limit = 10000

required_length = 3

Output: 147

Explanation:
When the limit is 10000 and required chain length is 3, there are 147 numbers whose chain becomes 3 in length


## Requirements

* The inputs are valid positive integers.
* The factorial of a digit is defined only for digits from 0 to 9.
* Python's integer arithmetic is sufficient to handle all intermediate values without overflow.
* The implementation must use memoization to cache chain lengths to avoid recomputation.
* The factorial values of digits must be precomputed before starting the iteration.
* The solution must avoid stack overflow by using an iterative approach instead of deep recursion.
* Efficient handling of duplicate work using a dictionary or array is expected.
* Function must be tested for correctness and performance.

## Function Signature

```python
def count_digit_factorial_chains(limit: int, required_length: int) -> int:
    pass
```

## Edge Cases

* Starting with a number that is equal to the sum of the factorial of its digits must terminate immediately with chain length one.
* A starting number that maps directly into a known loop must be terminated at the first repetition, even if the loop is longer.
* If a number starts repeating immediately (such as 145), the chain length should be considered one, not zero.
* If the same chain is formed by multiple numbers, each such number should be counted independently.

## Constraints

* The function must not use recursion that exceeds Python's default recursion depth.
* The function must terminate in a reasonable time when `limit` is up to one million.
* You must not use any libraries outside of the standard library.
* You must precompute the factorials of digits 0 through 9 only once and reuse them efficiently.
* The chain must stop at the first occurrence of a repeated value. Any values after the first repeated value must not be counted.
* Each digit factorial sum must be computed accurately without integer overflow.
* Chains must be computed independently for each starting number.
* Intermediate values can be large, so memoization or caching is encouraged.
* If a starting number itself falls into a loop immediately, its chain length is considered zero.

In [ ]:
# code

import math

def count_digit_factorial_chains(limit: int, required_length: int) -> int:
    # Precompute factorials of digits 0 to 9
    digit_factorials = [math.factorial(i) for i in range(10)]

    # Cache to memoize chain lengths
    chain_length_cache = {}

    def next_term(n: int) -> int:
        """Returns the sum of the factorial of the digits of n"""
        total = 0
        while n > 0:
            total += digit_factorials[n % 10]
            n //= 10
        return total

    def compute_chain_length(start: int) -> int:
        """Computes the non-repeating chain length for a given starting number"""
        visited = []
        seen = set()
        current = start

        while current not in seen:
            if current in chain_length_cache:
                # Use cached result
                total_length = len(visited) + chain_length_cache[current]
                break
            seen.add(current)
            visited.append(current)
            current = next_term(current)
        else:
            # A loop was encountered
            total_length = len(visited)

        # Cache the results for all unique numbers in this chain
        for index, value in enumerate(visited):
            if value not in chain_length_cache:
                chain_length_cache[value] = total_length - index

        return total_length

    # Main loop to count how many chains have exactly the required length
    count = 0
    for n in range(1, limit):
        if compute_chain_length(n) == required_length:
            count += 1

    return count

print(count_digit_factorial_chains(10000, 3)) # 147
print(count_digit_factorial_chains(10000, 5)) # 57
print(count_digit_factorial_chains(1000000, 60)) # 402

147
57
402


In [ ]:
# tests

import unittest
from math import factorial
from main import count_digit_factorial_chains

class TestCountDigitFactorialChains(unittest.TestCase):

    def test_example_from_prompt(self):
        self.assertEqual(count_digit_factorial_chains(10000, 3), 147)

    def test_single_value_limit(self):
        result = count_digit_factorial_chains(2, 1)
        self.assertIn(result, [0, 1])

    def test_small_limit_no_match(self):
        self.assertEqual(count_digit_factorial_chains(10, 5), 0)

    def test_chain_length_1(self):
        result = count_digit_factorial_chains(146, 1)
        self.assertGreaterEqual(result, 1)

    def test_immediate_loop_edge_case(self):
        self.assertIn(169, [169])
        self.assertGreaterEqual(count_digit_factorial_chains(170, 3), 1)

    def test_large_limit_with_known_result(self):
        result = count_digit_factorial_chains(1_000_000, 60)
        self.assertEqual(result, 402)

    def test_multiple_matches(self):
        self.assertGreaterEqual(count_digit_factorial_chains(100, 1), 1)

    def test_chain_length_2(self):
        self.assertGreaterEqual(count_digit_factorial_chains(872, 2), 1)

    def test_edge_limit_exclusion(self):
        result_inclusive = count_digit_factorial_chains(100, 1)
        result_exclusive = count_digit_factorial_chains(99, 1)
        self.assertLessEqual(result_exclusive, result_inclusive)

    def test_non_trivial_chain(self):
        self.assertIsInstance(count_digit_factorial_chains(70, 5), int)

    def test_shared_chains_counted_independently(self):
      count = count_digit_factorial_chains(873, 2)
      self.assertGreaterEqual(count, 2)

    def test_starting_number_in_known_loop_has_zero_length(self):
      count = count_digit_factorial_chains(169, 1)
      self.assertGreaterEqual(count, 1)

test_chain_length_1 (__main__.TestCountDigitFactorialChains.test_chain_length_1) ... ok
test_chain_length_2 (__main__.TestCountDigitFactorialChains.test_chain_length_2) ... ok
test_edge_limit_exclusion (__main__.TestCountDigitFactorialChains.test_edge_limit_exclusion) ... ok
test_example_from_prompt (__main__.TestCountDigitFactorialChains.test_example_from_prompt) ... ok
test_immediate_loop_edge_case (__main__.TestCountDigitFactorialChains.test_immediate_loop_edge_case) ... ok
test_large_limit_with_known_result (__main__.TestCountDigitFactorialChains.test_large_limit_with_known_result) ... ok
test_multiple_matches (__main__.TestCountDigitFactorialChains.test_multiple_matches) ... ok
test_non_trivial_chain (__main__.TestCountDigitFactorialChains.test_non_trivial_chain) ... ok
test_shared_chains_counted_independently (__main__.TestCountDigitFactorialChains.test_shared_chains_counted_independently) ... ok
test_single_value_limit (__main__.TestCountDigitFactorialChains.test_single_value_li

# Model Breaking Proof

#### Model Breaking Task URL: https://labeling-ms.turing.com/conversations/46078/view

#### Model code:

```python
def count_digit_factorial_chains(limit: int, required_length: int) -> int:
    # Precompute the factorials of the digits from 0 to 9
    factorials = [1] * 10
    for i in range(2, 10):
        factorials[i] = factorials[i - 1] * i

    # Dictionary for memoization of chain lengths
    chain_length_cache = {}

    def digit_factorial_sum(n: int) -> int:
        """Return the sum of factorials of the digits of n."""
        return sum(factorials[int(d)] for d in str(n))

    result_count = 0

    for start in range(1, limit):
        visited = []
        n = start

        # <Issue>: Model adds cached chain length to current visited length without checking if overlap causes overcount
        while n not in visited:
            if n in chain_length_cache:
                length_of_chain = chain_length_cache[n] + len(visited)
                break

            visited.append(n)
            n = digit_factorial_sum(n)
        
        # <Issue>: Model caches partial chain based only on repetition point, resulting in wrong count
        # Length of the current non-repeating chain
        if n in visited:
            index_of_repetition = visited.index(n)
            length_of_chain = len(visited)
            chain_length_cache[start] = length_of_chain - index_of_repetition

        # Check if the chain length for this start is exactly the required_length
        if length_of_chain == required_length:
            result_count += 1

        # <Issue>: Model applies wrong offset for early chain steps due to prior overcount
        # Cache the length of the chain for all visited numbers
        for i, num in enumerate(visited):
            if num not in chain_length_cache:
                chain_length_cache[num] = length_of_chain - i

    return result_count

print(count_digit_factorial_chains(10000, 3)) # 147 #received 151
print(count_digit_factorial_chains(10000, 5)) # 57 #received 62
print(count_digit_factorial_chains(1000000, 60)) # 402
```